In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/content/emails.csv")

In [3]:
df.tail()

,text,spam
5723,Subject: re : research and development charges...,0
5724,"Subject: re : receipts from visit jim , than...",0
5725,Subject: re : enron case study update wow ! a...,0
5726,"Subject: re : interest david , please , call...",0
5727,Subject: news : aurora 5 . 2 update aurora ve...,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5728 entries, 0 to 5727
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5728 non-null   object
 1   spam    5728 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 89.6+ KB


In [5]:
import re
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')


nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
for i in range(5728):
    df['text'][i] = df['text'][i][9:]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
for i in range(5728):
    sentence = df['text'][i]
    sentence = re.sub(r'[^\w\s]', '', sentence)
    new = ''
    for word in sentence.split(' '):
        if word not in stop:
            new = new + ' ' + str(lemmatizer.lemmatize(word)).lower()
    
    new = re.sub(r"(^|\W)\d+", '', new)

    df['text'][i] = new

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [13]:
df['text']

0        naturally irresistible corporate identity  lt...
1        stock trading gunslinger  fanny merrill muzo ...
2        unbelievable new home made easy  im wanting s...
3        color printing special  request additional in...
4        money  get software cd   software compatibili...
                              ...                        
5723      research development charge gpg             ...
5724      receipt visit  jim   thanks invitation visit...
5725      enron case study update  wow  day   super  t...
5726      interest  david   please  call shirley crens...
5727     news  aurora  update  aurora version    faste...
Name: text, Length: 5728, dtype: object

In [17]:
#index of all words in vocabulary
word_dict = {}
ind = 0
for i in range(5728):
    sentence = df['text'][i]
    for word in sentence.split(' '):
        if word not in word_dict:
            word_dict[word] = ind
            ind += 1

In [19]:
len(word_dict.values())

30991

In [20]:
new_list = []
label_list = []
for i in range(5728):
    sentence = df['text'][i]
    a_list = [0]*30991
    for word in sentence.split(' '):
        word_index = word_dict[word]
        a_list[word_index] += 1
    new_list.append(a_list)
    label_list.append(int(df['spam'][i]))

In [23]:
new_list = np.array(new_list)
new_list.shape

(5728, 30991)

In [24]:
label = np.array(label_list)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new_list, label)

In [26]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [29]:
y_pred = clf.predict(X_test)

In [27]:
clf.score(X_test, y_test)

0.9895251396648045

In [34]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

In [35]:
confusion_matrix(y_test, y_pred)

array([[1079,    1],
       [  14,  338]])

In [31]:
precision_score(y_test, y_pred)

0.9970501474926253

In [33]:
recall_score(y_test, y_pred)

0.9602272727272727